2. Using a PyTorch Model (Custom Model)
If you want to experiment with a pre-trained PyTorch-based model (like ResNet), you can implement your custom FER pipeline. This involves:

Using MTCNN for face detection.
Passing the cropped face to a pre-trained PyTorch model for emotion classification.
Setup PyTorch Model
For simplicity, you can use a pre-trained FER model or fine-tuned ResNet.

Code for PyTorch Model

In [ ]:
import cv2
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True, device='cpu')

# Load a pre-trained ResNet for emotion classification
# Replace with your fine-tuned FER model if available
model = resnet18(pretrained=True)
model.fc = torch.nn.Linear(model.fc.in_features, 7)  # Assuming 7 emotion classes
model.eval()

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Preprocessing pipeline for PyTorch model
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detect faces using MTCNN
    boxes, _ = mtcnn.detect(frame)

    if boxes is not None:
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = frame[y1:y2, x1:x2]  # Crop the face region

            # Ensure the face region is valid
            if face.size == 0:
                continue

            # Preprocess the face for the model
            face_tensor = transform(face).unsqueeze(0)  # Add batch dimension

            # Predict emotion
            with torch.no_grad():
                outputs = model(face_tensor)
                _, predicted = torch.max(outputs, 1)
                emotion = emotion_labels[predicted.item()]

            # Draw bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Add emotion label
            cv2.putText(frame, emotion, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the video feed
    cv2.imshow("PyTorch Emotion Detection", frame)

    # Exit on 'q' key or if window is closed
    key = cv2.waitKey(1)
    if key == ord('q') or cv2.getWindowProperty("PyTorch Emotion Detection", cv2.WND_PROP_VISIBLE) < 1:
        break

cap.release()
cv2.destroyAllWindows()
